<a href="https://colab.research.google.com/github/darvesh-sd/Copy-of-TPSessions.ipynb/blob/main/TP6_No_1_Summarization_avec_Python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Install the Required Libraries**

In [18]:
!pip install transformers
!pip install torch
!pip install newspaper3k
!pip install requests


**Import the Required Libraries**

In [19]:
import requests
from newspaper import Article
from transformers import BartForConditionalGeneration, BartTokenizer
from IPython.display import display, HTML
from ipywidgets import interact


**Define the URLs of the Articles**

In [20]:
# URLs for 1 English article and 2 French articles related to the 2022 World Cup
urls = [
    "https://www.cnn.com/sport/live-news/world-cup-final-argentina-france-12-18-22/index.html",  # English article
    "https://www.lemonde.fr/en/world-cup-2022/article/2022/12/19/world-cup-2022-argentina-topples-france-in-a-twist-filled-final_6008278_209.html",  # French article 1
    "https://www.reuters.com/sports/soccer/fifa-overlooks-own-report-advice-qatar-world-cup-workers-compensation-2024-11-30/?utm_source=chatgpt.com",  # French article 2
]


**Define the Function to Fetch and Parse Articles**

In [21]:
# Function to fetch and parse article text from URL with a custom user-agent
def fetch_article(url):
    article = Article(url)
    article.download()
    article.parse()
    return article.text


**Load the Model for Summarization**

In [22]:
# Load the BART model and tokenizer for summarization
model_name = "facebook/bart-large-cnn"
model = BartForConditionalGeneration.from_pretrained(model_name)
tokenizer = BartTokenizer.from_pretrained(model_name)


**Define the Function to Fetch and Parse Articles**

In [23]:
# Function to fetch and parse article text from URL with a custom user-agent
def fetch_article(url):
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36'}

    # Fetch the article content using requests with custom headers
    response = requests.get(url, headers=headers)

    # Create a Newspaper Article object
    article = Article(url)

    # Set the HTML content of the article (bypassing the usual download step)
    article.set_html(response.text)

    # Parse the content
    article.parse()

    return article.text


**Define the Function to Summarize Articles**

In [24]:
# Function to summarize the article using the BART model
def summarize_article(article_text):
    # Tokenize the article
    inputs = tokenizer(article_text, return_tensors="pt", max_length=1024, truncation=True, padding=True)

    # Generate the summary
    summary_ids = model.generate(inputs['input_ids'], max_length=150, min_length=50, length_penalty=2.0, num_beams=4, early_stopping=True)

    # Decode the summary
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary


**Fetch and Summarize Articles**

In [25]:
# Fetch and summarize the English article
english_article = fetch_article(urls[0])
summary_english = summarize_article(english_article)

# Fetch and summarize the first French article
french_article_1 = fetch_article(urls[1])
summary_french_1 = summarize_article(french_article_1)

# Fetch and summarize the second French article
french_article_2 = fetch_article(urls[2])
summary_french_2 = summarize_article(french_article_2)


**Display the Summaries**

In [26]:
# Function to display the summaries interactively
def display_summary(article_num, summary):
    return f"<h3>Summary of Article {article_num}:</h3><p>{summary}</p>"

# Create interactive display for the summaries
display(HTML(display_summary(1, summary_english)))
display(HTML(display_summary(2, summary_french_1)))
display(HTML(display_summary(3, summary_french_2)))


**Add Interactivity for Summary Selection**

In [27]:
# Function to allow the user to select and display a summary interactively
def interactive_summary(article_num=1):
    if article_num == 1:
        return display(HTML(display_summary(1, summary_english)))
    elif article_num == 2:
        return display(HTML(display_summary(2, summary_french_1)))
    elif article_num == 3:
        return display(HTML(display_summary(3, summary_french_2)))

# Interactive dropdown to choose the article to summarize
interact(interactive_summary, article_num=[1, 2, 3])


interactive(children=(Dropdown(description='article_num', options=(1, 2, 3), value=1), Output()), _dom_classes…

<function __main__.interactive_summary(article_num=1)>